In [3]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.time import RandomActivationByType
from mesa.datacollection import DataCollector
from numpy import random
from mesa import Agent
from numpy import random
from mesa.visualization import CanvasGrid, ModularServer
import numpy as np

In [10]:
def get_distance(pos1, pos2):
    return (pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2

class Cell(Agent):

    def __init__(self, unique_id, model, capacities):
        super().__init__(unique_id, model)
        # Initialize sugar and spice
        self.capacities = capacities
        self.sugar = capacities[0]
        self.spice = capacities[1]

    def step(self):
        # Move agent
        self.regenerate()

    def regenerate(self):
        # Regenerate sugar
        self.sugar = min(self.sugar + 1, self.capacities[0])
        self.spice = min(self.spice + 1, self.capacities[1])

class Trader(Agent):
    def __init__(self, unique_id, model, sugar, sugar_metabolism,
                 spice, spice_metabolism, vision):
        super().__init__(unique_id, model)

        # Set initial parameters
        self.sugar = sugar
        self.sugar_metabolism = sugar_metabolism
        self.spice = spice
        self.spice_metabolism = spice_metabolism
        self.vision = vision

        # Weight for both sugar and spice when moving
        self.spice_weight = sugar_metabolism / (sugar_metabolism + spice_metabolism)
        self.sugar_weight = 1 - self.spice_weight

    def step(self):
        # Move agent
        self.move()

        # Pick up sugar and spice
        self.pick_up()

        # Trade sugar and spice
        self.trade()

         # Metabolize sugar and spice
        self.metabolize()

    def move(self):

         # Get neighborhood
        neighbors = [i for i in self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False, radius=self.vision)]
        
        # Get cell with most sugar
        max_total = -1
        shortest_distance = 100
        max_cell = []
        for neighbor in neighbors:
            this_cell = self.model.grid.get_cell_list_contents([neighbor])
            for agent in this_cell:
                if isinstance(agent, Cell):

                    # Compute weighted average of sugar and spice
                    weighted_sugar = self.sugar_weight * agent.sugar
                    weighted_spice = self.spice_weight * agent.spice
                    total = weighted_sugar + weighted_spice

                    # Update max_sugar and max_sugar_cells
                    if total > max_total:
                        distance = get_distance(self.pos, neighbor)
                        shortest_distance = distance
                        max_total = total
                        max_cell = [neighbor]


                    # Append to max_sugar_cells if equal
                    elif total == max_total:
                        distance = get_distance(self.pos, neighbor)
                        if distance < shortest_distance:
                            shortest_distance = distance
                            max_cell = [neighbor]
                        elif distance == shortest_distance:
                            max_cell.append(neighbor)

        # Move to cell with most sugar
        new_position = random.choice(range(len(max_cell)))
        new_position = max_cell[new_position]
        self.model.grid.move_agent(self, new_position)

    def pick_up(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        # Grab all sugar and spice from cell
        for agent in this_cell:
            if isinstance(agent, Cell):
                self.sugar += agent.sugar
                agent.sugar = 0

                self.spice += agent.spice
                agent.spice = 0

    def metabolize(self):
        # Metabolize sugar
        self.sugar -= self.sugar_metabolism

        # Metabolize spice
        self.spice -= self.spice_metabolism

        # Die if sugar is less than 0
        if self.sugar < 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)

            
        # Die if spice is less than 0
        if self.spice < 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)

    def trade(self):
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False, radius=1)
        for neighbor in neighbors:
            if isinstance(neighbor, Trader):
                my_mrs = self.get_mrs_sugar_spice()
                their_mrs = neighbor.get_mrs_sugar_spice()
                if my_mrs == their_mrs:
                    continue

                if my_mrs > their_mrs:
                    trader_high_mrs = self
                    trader_low_mrs = neighbor
                else:
                    trader_high_mrs = neighbor
                    trader_low_mrs = self

                trade_price = np.sqrt(my_mrs * their_mrs)

                if trade_price > 1:
                    trade_spice = trade_price
                    trade_sugar = 1
                else:
                    trade_spice = 1
                    trade_sugar = 1 / trade_price

                trade_sugar = min(trade_sugar, trader_low_mrs.sugar)
                trade_spice = min(trade_spice, trader_high_mrs.spice)

                if (trade_sugar > 0 and trade_spice > 0 and
                        self.improve_welfare(trader_high_mrs, trader_low_mrs, trade_sugar, trade_spice)):
                    trader_high_mrs.spice -= trade_spice
                    trader_high_mrs.sugar += trade_sugar
                    trader_low_mrs.spice += trade_spice
                    trader_low_mrs.sugar -= trade_sugar

    def get_mrs_sugar_spice(self):
        return (self.sugar_metabolism * self.spice) / (self.spice_metabolism * self.sugar)

    def improve_welfare(self, trader_high_mrs, trader_low_mrs, trade_sugar, trade_spice):
        high_mrs_after_trade = (trader_high_mrs.sugar_metabolism * (trader_high_mrs.spice - trade_spice)) / (
                    trader_high_mrs.spice_metabolism * (trader_high_mrs.sugar + trade_sugar))
        low_mrs_after_trade = (trader_low_mrs.sugar_metabolism * (trader_low_mrs.spice + trade_spice)) / (
                    trader_low_mrs.spice_metabolism * (trader_low_mrs.sugar - trade_sugar))
        
        improves_welfare = high_mrs_after_trade < trader_high_mrs.get_mrs_sugar_spice() and low_mrs_after_trade > trader_low_mrs.get_mrs_sugar_spice()
        mrs_no_crossing = high_mrs_after_trade > low_mrs_after_trade

        return improves_welfare and mrs_no_crossing



In [5]:

class SugarScape(Model):
    def __init__(self, height=50, width=50, initial_population=100):
        super().__init__()
        # Set parameters
        self.height = height
        self.width = width
        self.initial_population = initial_population

        self.schedule = RandomActivationByType(self)
        self.grid = MultiGrid(self.height, self.width, False)

        self.datacollector = DataCollector({"Agents": lambda m: m.schedule.get_type_count(Trader)})

        # Create cells
        id = 0
        for content, (x, y) in self.grid.coord_iter():
            # Instantiate cell
            capacities = random.randint(1, 10, 2)
            cell = Cell(id, self, capacities)

            # Place cell on grid
            self.grid.place_agent(cell, (x, y))
            self.schedule.add(cell)

            # Increment id
            id += 1

        # Create traders
        for i in range(self.initial_population):

            # Random position
            x = random.randint(0, self.width)
            y = random.randint(0, self.height)

            # Instantiate trader
            sugar, spice = random.randint(1, 10, 2)
            sugar_metabolism, spice_metabolism = random.randint(1, 4, 2)
            vision = random.randint(1, 4)
            trader = Trader(id, self, sugar, sugar_metabolism, spice, spice_metabolism, vision)

            # Place trader on grid
            self.grid.place_agent(trader, (x, y))
            self.schedule.add(trader)

            # Increment id
            id += 1

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        self.running = self.schedule.get_agent_count() > 0

    def run_model(self, step_count=200):
        for i in range(step_count):
            self.step()


In [12]:


def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {"Filled": "true",
                 "r": 0.5,
                 "w": 1,
                 "h": 1}

    if type(agent) is Trader:
        portrayal["Color"] = "red"
        portrayal["Layer"] = 1
        portrayal["Shape"] = "circle"
    elif type(agent) is Cell:
        portrayal["Shape"] = "rect"
        portrayal["Color"] = "green" if agent.sugar > 0 and agent.spice > 0 else "black"
        portrayal["Layer"] = 0

    return portrayal

canvas_element = CanvasGrid(agent_portrayal, 50, 50, 500, 500)

server = ModularServer(
    SugarScape, [canvas_element], "Sugarscape Model",
    {"height": 50, "width": 50, "initial_population": 100}
)
server.port = 8569
server.launch()


Interface starting at http://127.0.0.1:8569


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}


{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step","step":33}
{"type":"get_step",

In [8]:
SugarScape().run_model()